# Read Patient1 IMC file

version 2024-02-06 A.L.

In [1]:
import numpy as np

In [2]:
!ls -l ../data

total 1296688
drwxr-xr-x  14 arvid  staff        448 Oct 30  2020 20201023_pilotTMA
-rw-rw-r--@  1 arvid  staff  308296880 Nov 16  2021 Patient1.mcd
-rw-rw-r--@  1 arvid  staff  118532798 Nov 16  2021 Patient1_pos1_1_1.txt
-rw-rw-r--@  1 arvid  staff  118427612 Nov 16  2021 Patient1_pos1_2_2.txt
-rw-rw-r--@  1 arvid  staff  118634929 Nov 16  2021 Patient1_pos1_3_3.txt
-rw-r--r--@  1 arvid  staff       3198 Jan 29 10:03 panel.csv
-rw-r--r--@  1 arvid  staff         83 Jan 29 09:59 sample_metadata.csv


In [3]:
from readimc import MCDFile, TXTFile

In [4]:
mcd_fn = "../data/Patient1.mcd"

In [5]:
with MCDFile(mcd_fn) as f:
    num_slides = len(f.slides)

print(f"Number of slides: {num_slides}")
f

Number of slides: 1


../data/Patient1.mcd

### Extracting metadata

In [6]:
with MCDFile(mcd_fn) as f:
    # first slide
    slide = f.slides[0]
    print(f'first slide: {slide.id}, {slide.description}, {slide.width_um}, {slide.height_um}')

    # first panorama of first slide
    panorama = slide.panoramas[0]
    print(f'first panorama of first slide: {panorama.id}, {panorama.description}, {panorama.width_um}, {panorama.height_um}')

    # first acquisition of first slide
    acquisition = slide.acquisitions[0]
    print(f'\nfirst acquisition of first slide: {acquisition.id}, {acquisition.description}, {acquisition.width_um},{acquisition.height_um}')
    print(f'channel names (metals): {acquisition.channel_names}')
    print(f'channel labels (targets): {acquisition.channel_labels}')

    # second acquisition of first slide
    acquisition = slide.acquisitions[1]
    print(f'\nsecond acquisition of first slide: {acquisition.id}, {acquisition.description}, {acquisition.width_um},{acquisition.height_um}')
    print(f'channel names (metals): {acquisition.channel_names[1]}')
    print(f'channel labels (targets): {acquisition.channel_labels[1]}')

first slide: 0, Slide, 75000.0, 25000.0
first panorama of first slide: 1, wflow2.jpg, 75000.0, 24910.0

first acquisition of first slide: 1, pos1_1, 600.0,600.0
channel names (metals): ['ArAr80', 'Y89', 'In113', 'In115', 'Xe131', 'Xe134', 'Ba136', 'La138', 'Pr141', 'Nd142', 'Nd143', 'Nd144', 'Nd145', 'Nd146', 'Sm147', 'Nd148', 'Sm149', 'Nd150', 'Eu151', 'Sm152', 'Eu153', 'Sm154', 'Gd155', 'Gd156', 'Gd158', 'Tb159', 'Gd160', 'Dy161', 'Dy162', 'Dy163', 'Dy164', 'Ho165', 'Er166', 'Er167', 'Er168', 'Tm169', 'Er170', 'Yb171', 'Yb172', 'Yb173', 'Yb174', 'Lu175', 'Yb176', 'Ir191', 'Ir193', 'Pt196', 'Pb206']
channel labels (targets): ['80ArAr', 'Myelope_276((2967))Y89', 'Histone_126((2979))In113', 'SMA_174((2780))In115', '131Xe', '134Xe', '136Ba', '138La', 'CD16_1820((2945))Pr141', 'CD38_1719((2961))Nd142', 'HLA-DR_1849((2953))Nd143', 'CD27_2231((2946))Nd144', 'CD15_627((2954))Nd145', 'CD45RA_732((2955))Nd146', 'CD163_1863((2947))Sm147', 'Beta-2M_1855((2962))Nd148', 'CD20_36((2980))Sm149', 'CD

### Reading slide images

IMC .mcd files can store slide images uploaded by the user (e.g., photographs) or acquired by the instrument. For supported image file formats, these images can be read as follows:

In [7]:
with MCDFile(mcd_fn) as f:
    panorama = f.slides[0].panoramas[0]  # first panorama of first slide
    img = f.read_panorama(panorama)  # numpy array

print(img.shape, img.dtype)

(793, 2389, 4) uint8


### Reading panorama images

IMC .mcd files can contain zero or more panorama images acquired by the instrument, which can be read as follows:

In [8]:
with MCDFile(mcd_fn) as f:
    panorama = f.slides[0].panoramas[0]  # first panorama of first slide
    img = f.read_panorama(panorama)  # numpy array

print(img.shape, img.dtype)

(793, 2389, 4) uint8


### Reading IMC acquisitions

IMC .mcd files can contain zero or more IMC acquisitions, which can be read as follows:

In [9]:
with MCDFile(mcd_fn) as f:
    acquisition = f.slides[0].acquisitions[0]  # first acquisition of first slide
    img = f.read_acquisition(acquisition)  # array, shape: (c, y, x), dtype: float32

print(img.shape, img.dtype)

(47, 600, 600) float32


### Reading before/after-ablation images

The IMC instrument may be configured to acquire an optical image before/after each IMC acquisition. If available, these before/after-ablation images can be read as follows:

In [10]:
with MCDFile(mcd_fn) as f:
    acquisition = f.slides[0].acquisitions[0]  # first acquisition of first slide
    before_ablation_img = f.read_before_ablation_image(acquisition)  # array or None
    after_ablation_img = f.read_after_ablation_image(acquisition)  # array or None

print(before_ablation_img)
print(after_ablation_img)

None
None


## Visualize IMC data using napari / napari-imc

- Multi-dimensional image viewer for Python
- Over 250 plugins available via https://www.napari-hub.org 
- Natively reads TIFF files; reads MCD/HDF5/Zarr/OME-NGFF via plugins
- “Scriptable”: napari can be controlled using Python (e.g. Jupyter notebooks)


See:
- https://napari.org/stable/tutorials/fundamentals/getting_started.html

- https://docs.google.com/presentation/d/1IUnl2lz3iU_D_0grOAbYyBeVrPQDU1dI-RHlX0dt4ko/edit#slide=id.p

In [11]:
import napari

# import sample data
from skimage import data
from skimage.data import cells3d


In [12]:
# viewer = napari.view_image(data.moon())

In [78]:
# create a `Viewer` and `Image` layer here
# viewer, image_layer = napari.imshow(cells3d())

In [13]:
?napari.view_image


Signature:
napari.view_image(
    data=None,
    *,
    channel_axis=None,
    rgb=None,
    colormap=None,
    contrast_limits=None,
    gamma=1,
    interpolation2d='nearest',
    interpolation3d='linear',
    rendering='mip',
    depiction='volume',
    iso_threshold=None,
    attenuation=0.05,
    name=None,
    metadata=None,
    scale=None,
    translate=None,
    rotate=None,
    shear=None,
    affine=None,
    opacity=1,
    blending=None,
    visible=True,
    multiscale=None,
    cache=True,
    plane=None,
    experimental_clipping_planes=None,
    custom_interpolation_kernel_2d=None,
    title='napari',
    ndisplay=2,
    order=(),
    axis_labels=(),
    show=True,
) -> napari.viewer.Viewer
Docstring:
Create a viewer and add an image layer.

Parameters
----------
data : array or list of array
    Image data. Can be N >= 2 dimensional. If the last dimension has length
    3 or 4 can be interpreted as RGB or RGBA if rgb is `True`. If a
    list and arrays are decreasing in

In [14]:
with MCDFile(mcd_fn) as f:
    acquisition = f.slides[0].acquisitions[0]  # first acquisition of first slide
    img = f.read_acquisition(acquisition)  # array, shape: (c, y, x), dtype: float32

print(img.shape, img.dtype)

chn = 9
min_value = np.min(img[chn])
max_value = np.max(img[chn])
contrast_stretched_image = (img[chn] - min_value) / (max_value - min_value)

viewer = napari.view_image(contrast_stretched_image, gamma=1.0, 
                           title=f"{mcd_fn} - chn:{chn}, {acquisition.description}, {img[chn].shape}, name (metal): {acquisition.channel_names[chn]}, label (target): {acquisition.channel_labels[chn]} ")
# viewer.layers.selection.active.mode = 'transform'


(47, 600, 600) float32
